# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x1e44fa27100>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
3028,1.9,969.75,-200,11.228615,1022.50,134.0,787.00,123.0,1552.00,1073.50,26.350,27.325000,0.923818
8803,0.3,1083.25,-200,5.838871,804.25,208.1,702.25,128.2,1104.50,615.25,18.000,35.375000,0.723328
4474,-200.0,875.25,-200,4.041180,713.00,-200.0,1026.75,-200.0,1435.50,809.25,19.450,73.424999,1.641728
8249,1.2,990.00,-200,3.897128,705.00,216.5,872.75,138.5,913.25,771.50,9.025,41.450000,0.477541
2197,2.9,1226.50,-200,19.850891,1292.25,214.0,660.75,104.0,2087.00,1326.50,21.725,53.925000,1.384784


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [6]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [7]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [8]:
np.random.seed(66398)                 # 66398 is the number last five digits of my student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:
data_training_unproc.sample(5)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6667,4.1,1409.75,22.430899,1362.50,696.0,690.00,187.0,1514.75,2014.00,13.875,42.849999,0.676103
2811,1.8,1057.50,10.707879,1003.75,92.0,776.00,98.0,1683.50,947.25,28.775,32.825001,1.277586
28,1.5,964.50,4.707072,748.50,94.0,1325.25,85.0,1332.50,821.00,8.175,63.400000,0.690484
6851,0.3,716.00,0.749887,472.50,54.0,1595.25,41.0,734.25,342.75,4.725,41.400001,0.357609
8945,3.1,1323.25,14.862431,1144.25,457.3,502.25,186.7,1405.25,1568.25,18.800,33.450001,0.718621


In [21]:
# Write your code here

ndata_train, ncols_train = np.shape(data_training_unproc) #find the size of training data
train_process_mean=np.empty(ncols_train)#define an array to store mean values to replace -200
train_feature_mean=np.empty(ncols_train)#define an array to store feature mean after processing
train_feature_std=np.empty(ncols_train)#define an array to store standard deviations
index_train=range(ndata_train)
data_training_proc=data_training_unproc.copy() #copy the unprocessing train data, store processed training data

#design a function to replace missing values
def data_replace(data,mean):
    for i in range(ncols_train):
        data.iloc[:,i].replace(-200,mean[i],inplace=True) 

#design a function to standard the data
def standard(data,mean,std):
    for i in range(ncols_train):
        data.iloc[:,i]=(data.iloc[:,i]-mean[i])/std[i]
    
#get the mean and store in train_process_mean
for i in range(ncols_train):  
    train_process_mean[i]=data_training_proc[data_training_proc.iloc[:,i]!=-200].iloc[:,i].mean() 

#using function replace the missing values
data_replace(data_training_proc,train_process_mean)

#get the std and store in train_feature_std
for i in range(ncols_train):
    train_feature_std[i]=np.std(data_training_proc.iloc[:,i])

#using function to standard data
standard(data_training_proc,train_process_mean,train_feature_std)


## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [22]:
# Write your code here
yTrain=np.array(data_training_proc.iloc[:, 0].values).reshape(-1,1)
X=np.array(data_training_proc.iloc[:, 1:].values)
ones=np.ones_like(yTrain)#make a array with 1, shape is like yTrain
XTrain=np.column_stack((ones,X))#combine ones and X, we will use this to do the project

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [20]:
# Write your code here
data_val_proc=data_val_unproc.copy()
data_replace(data_val_proc,train_process_mean)  #replace the missing value
standard(data_val_proc,train_process_mean,train_feature_std)  #standard the data

data_val_proc

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
6109,-0.662469,-1.084492,-0.599700,-0.537494,0.176371,0.422416,0.448214,-1.450434,-0.020148,-1.252717,-0.569525,-1.534046
6899,-0.730812,-1.429015,-1.196287,-1.526108,-0.717311,1.533999,-0.807769,-1.881003,-0.670404,-1.581665,0.069429,-1.481653
6528,0.636033,0.413484,0.530447,0.669747,0.485357,-0.868584,0.001169,0.082418,0.413778,-0.283036,0.511445,0.025683
5807,-0.730812,-1.362671,-0.864794,-0.914337,-0.712557,0.979711,-1.276102,-1.231143,-1.289032,-0.214386,-0.878791,-0.971591
7854,-0.594127,-0.352381,-0.756229,-0.752285,-0.271896,-0.041662,0.654706,-1.548059,0.680711,-1.561642,0.011077,-1.490202
...,...,...,...,...,...,...,...,...,...,...,...,...
6183,2.412932,2.165196,2.342985,2.080273,3.437361,-1.504061,0.959122,1.172317,1.901522,-0.909467,1.660979,-0.121965
2746,-1.209207,-1.328917,-1.036037,-1.201045,-1.092848,1.019804,-1.574132,-0.236683,-1.202374,0.537903,-0.357999,0.423676
5796,-1.209207,-1.659473,-1.215109,-1.570217,-1.012036,2.069243,-1.787010,-1.306183,-1.516116,-0.268734,0.137992,-0.239769
5520,5.009937,3.699254,5.668690,4.046954,3.304259,-1.995202,1.384879,3.855537,3.349417,0.532182,0.721512,1.685428


In [24]:
# Write your code here
data_val_proc=data_val_unproc.copy()
data_replace(data_val_proc,train_process_mean)  #replace the missing value
standard(data_val_proc,train_process_mean,train_feature_std)  #standard the data

#XValid and yValid
yValid=np.array(data_val_proc.iloc[:, 0].values).reshape(-1,1)
X=np.array(data_val_proc.iloc[:, 1:].values)
ones=np.ones_like(yValid)
XValid=np.hstack((ones,X))

#define the grid of values for para
learn_rate=np.logspace(-5,-1,num=5,base=10)
reg_para=np.logspace(-2,2,num=5,base=10)
batch_size=np.linspace(500,2000,num=5)

#define the value of w at first
w_test=(np.ones((12,1)))

#define a function to get the RMSE
def get_RMSE(w,y,X,S):
    f_x=np.dot(w.T,X.T)
    resid=(y-f_x.T)
    RMSE=np.sqrt((np.dot(resid.T,resid))/S)
    return RMSE
    
#define a function to get random batch of data
def get_random(S):
    index_val = np.random.permutation(data_training_proc.shape[0])
    X=XTrain[index_val[0:int(S)]]
    y=yTrain[index_val[0:int(S)]] 
    return X,y

#define a function to get w
def get_w(w_old,theta,lamba,S):
    i=0
    while i<=200:
        X,y=get_random(S)
        w_1=(1-theta*lamba)*w_old
        resid=y-np.dot(X,w_old)
        w_2=((2*theta)/S)*np.dot(X.T,resid)
        w_old=w_1+w_2
        i+=1
    return w_old

#create a dict to store the best param later
dict={'i':0,'k':0,'t':0}
min=None


#use the for loop to get the best param
for i in range(5):
    for k in range(5):
        for t in range(5):
            w=get_w(w_test,learn_rate[i],reg_para[k],batch_size[t])
            RMSE=get_RMSE(w,yValid,XValid,XValid.shape[0])
            #use this to replace the param with min rmse
            if min==None or min>RMSE:
                min=RMSE
                dict['i']=i
                dict['k']=k
                dict['t']=t

print(min)
#get the best param, store in each real name
theta=learn_rate[dict['i']]
lamba=reg_para[dict['k']]
S=batch_size[dict['t']]

print(theta)
print(lamba)
print(S)

[[0.35701887]]
0.1
0.1
1250.0


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [26]:
# Write your code here
data_training_new = air_quality.iloc[index[0:N+Nval], :].copy() # Select the new training data
ndata_train_new, ncols_train_new = np.shape(data_training_new) #find the size of training data
train_new_mean=np.empty(ncols_train_new)#define an array to store feature mean after processing
train_new_std=np.empty(ncols_train_new)#define an array to store standard deviations
index_train_new=range(ndata_train_new)
data_new_proc=data_training_new.copy() #copy the unprocessing train data, store processed training data


#get the mean and store in train_process_mean
for i in range(ncols_train_new):  
    train_new_mean[i]=data_new_proc[data_new_proc.iloc[:,i]!=-200].iloc[:,i].mean() 

#using function replace the missing values
data_replace(data_new_proc,train_new_mean)

#get the std
for i in range(ncols_train_new):
    train_new_std[i]=np.std(data_new_proc.iloc[:,i])

#using function to standard data
standard(data_new_proc,train_new_mean,train_new_std)

data_new_proc.sample(5)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
471,1.393785,2.066630,1.520329,1.489148,0.123182,-0.600641,0.745653,1.179357,1.618357,-0.166824,-1.199233,-1.185256
367,-0.519098,-0.411738,-0.756612,-0.754232,-0.841621,1.234516,-0.508150,-0.265921,-0.640703,-0.929386,0.724268,-0.643210
3042,-0.109195,-0.129893,0.233267,0.391467,-0.572044,-0.348933,0.320635,0.667928,-0.417446,1.801449,-1.439305,0.658330
802,-1.270588,-1.338796,-1.288703,-1.770284,-1.115928,2.976410,-1.846956,-0.809451,-1.725456,-0.771119,0.636437,-0.515325
8800,-1.065636,0.112353,-0.561937,-0.489175,-0.182812,-0.682872,0.314260,-0.772243,-0.764315,-0.316459,-0.038399,-0.431457


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [14]:
# Write your code here
ndata_test, ncols_test = np.shape(data_test_unproc) #find the size of training data
test_mean=np.empty(ncols_test)#define an array to store feature mean after processing
test_std=np.empty(ncols_test)#define an array to store standard deviations
index_test=range(ndata_test)
data_test_proc=data_test_unproc.copy() #copy the unprocessing train data, store processed training data


#get the mean and store in train_process_mean
for i in range(ncols_test):  
    test_mean[i]=data_test_proc[data_test_proc.iloc[:,i]!=-200].iloc[:,i].mean() 

#using function replace the missing values
data_replace(data_test_proc,test_mean)

#get the std
for i in range(ncols_test):
    test_std[i]=np.std(data_test_proc.iloc[:,i])

#using function to standard data
standard(data_test_proc,test_mean,test_std)

y_new=np.array(data_new_proc.iloc[:, 0].values).reshape(-1,1)
X=np.array(data_new_proc.iloc[:, 1:].values)
ones=np.ones_like(y_new)
X_new=np.column_stack((ones,X))

w_new=get_w(w_test,theta,lamba,S)

y_test=np.array(data_test_proc.iloc[:, 0].values).reshape(-1,1)
X=np.array(data_test_proc.iloc[:, 1:].values)
ones=np.ones_like(y_test)
X_test=np.column_stack((ones,X))
RMSE_test=get_RMSE(w_new,y_test,X_test,X_test.shape[0])

print(RMSE_test)

[[0.32359286]]


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [27]:
# Write your code here
#import the sklearn function we will use later
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#design a pipeline
data_preprocess = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = -200, strategy='mean')),#replace the missing value
    ('scaler', StandardScaler())
])


### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [28]:
# Write your code here
#use pipeline to fit and transform the training data
data_training_proc_2=data_preprocess.fit_transform(data_training_unproc)
#transform the validation data
data_val=data_preprocess.transform(data_val_unproc)

#divide the label and attributes to 2 array y and X
y=data_val[:,0]
X=data_val[:,1:]
y_test=data_training_proc_2[:,0]
X_test=data_training_proc_2[:,1:]



## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [29]:
# Write your code here
data_train_whole_unproc=air_quality.iloc[index[0:N+Nval], :].copy()

#design a pipeline to process whole data
data_preprocess_whole = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = -200, strategy='mean')),#replace the missing value to mean
    ('scaler', StandardScaler())
])
#fit and transform the whole data
data_train_whole=data_preprocess_whole.fit_transform(data_train_whole_unproc)
data_test=data_preprocess_whole.transform(data_test_unproc)#process the test data

#divide the labels and attributes 
y_train_whole=data_train_whole[:,0]
X_train_whole=data_train_whole[:,1:]

y_train_whole.reshape(-1,1)

y_test=data_test[:,0]
X_test=data_test[:,1:]

y_test.reshape(-1,1)
print(y_train_whole.shape)

(6523,)


In [30]:
# Write your code here
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV

#create test_fold to define which part is using to traing and which part is using to test 
test_fold = np.zeros((np.shape(data_train_whole)[0], 1))
test_fold[0:np.shape(X_test)[0]] = -1 #make sure the X_test part index is -1 so they will not used to train
ps = PredefinedSplit(test_fold)

#define the param choseing board
n_estimators=np.linspace(10,200,num=5,dtype=int)
max_features=np.linspace(0.01,1,num=5)
max_samples=np.linspace(0.01,1,num=5)

#build the model
forest_model = RandomForestRegressor()

#make a dict to store param, which used to GridsearchCV
parameters = {'n_estimators':n_estimators, 'max_features':max_features ,'max_samples':max_samples}

clf = GridSearchCV(forest_model, parameters,cv=ps)

#fit the model
clf_result=clf.fit(X_train_whole,y_train_whole)
best_params=clf.best_params_#get the best param

max_features=best_params['max_features']
max_samples=best_params['max_samples']
n_estimators=best_params['n_estimators']

print(best_params)
print(clf_result.best_score_)

{'max_features': 0.2575, 'max_samples': 1.0, 'n_estimators': 152}
0.9047496163404994


In [31]:
best_params['n_estimators']

152

### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [32]:
# Write your code here
data_train_whole_unproc=air_quality.iloc[index[0:N+Nval], :].copy()

#make a pipeline to preprocess the data
data_preprocess_whole = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = -200, strategy='mean')),
    ('scaler', StandardScaler())
])

#fit and transform the data
data_train_whole=data_preprocess_whole.fit_transform(data_train_whole_unproc)
data_test=data_preprocess_whole.transform(data_test_unproc)

#divide the data to labels and attributes
y_train_whole=data_train_whole[:,0]
X_train_whole=data_train_whole[:,1:]

y_train_whole.reshape(-1,1)

y_test=data_test[:,0]
X_test=data_test[:,1:]

y_test.reshape(-1,1)

#use the best param to build a ne model
final_forest_model = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features,max_samples=max_samples)

#fit and predict the data
final_forest_model.fit(X_train_whole,y_train_whole)
predict_y=final_forest_model.predict(X_test)

#use the function from sklearn to caculate rmse
from sklearn.metrics import mean_squared_error

error_mod = np.sqrt(mean_squared_error(y_test,predict_y))

print(error_mod)

0.24785687658684122
